In [1]:
import cadquery as cq
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show, 
                                       replay, enable_replay, disable_replay, reset_replay)
from jupyter_cadquery import set_sidecar, set_defaults
import numpy as np

show_object = show

Overwriting auto display for cadquery Workplane and Shape


In [2]:
set_sidecar("Gears")

In [3]:
enable_replay()
show_object = replay


Enabling jupyter_cadquery replay


In [4]:
set_defaults(axes=True, grid=True, axes0=True, edge_accuracy=0.001, timeit=True)

In [5]:
from math import pi, sin, cos, acos, tan, atan, ceil, radians, degrees, floor
import numpy as np
clearance = 0.05

In [6]:
def copier(solid, vect, number, distance, winkel):
    copies = []
    
    for i in range(number):
        copy = solid.rotate((0.0, 0.0, 0.0), (0.0, 0.0, 1.0), i * winkel)
        d = distance * i
        copy = copy.translate((vect[0] * d, vect[1] * d, vect[2] * d))
        copies.append(copy)
    
    return copies

def polar_to_cartesian(polvect):
    return (polvect[0] * cos(polvect[1]),
            polvect[0] * sin(polvect[1]))


#     Circle Involutes-Function:
#     Returns the Polar Coordinates of an Involute Circle
#     r = Radius of the Base Circle
#     rho = Rolling-angle in Degrees
def ev(r, rho):
    return (r / cos(rho), tan(rho) - rho)

def sphere_to_cartesian(vect):
    return (vect[0] * sin(vect[1]) * cos(vect[2]),  
            vect[0] * sin(vect[1]) * sin(vect[2]),
            vect[0] * cos(vect[1]))


In [ ]:

def rack(modul, length, height, width, pressure_angle=20.0, helix_angle=0.0):
    # Dimension Calculations
    pressure_angle = radians(pressure_angle)
    helix_angle = radians(helix_angle)
    
    modul = modul * (1.0 - clearance)
    c = modul / 6.0                                              # Tip Clearance
    mx = modul / cos(helix_angle)                                # Module Shift by Helix Angle in the X-Direction
    a = 2.0 * mx * tan(pressure_angle) + c * tan(pressure_angle) # Flank Width
    b = pi * mx / 2.0 - 2.0 * mx * tan(pressure_angle)           # Tip Width
    x = width * tan(helix_angle)                                 # Topside Shift by Helix Angle in the X-Direction
    nz = ceil((length + abs(2.0 * x)) / (pi * mx))               # Number of Teeth
    
    
    underside_points = [[0, -c],
                        [a, 2.0 * modul],
                        [a + b, 2.0 * modul],
                        [2.0 * a + b, -c],
                        [pi * mx, -c],
                        [pi * mx, modul - height],
                        [0.0, modul - height]]

    topside_points = [[0.0 + x, -c],
                      [a + x, 2.0 * modul],
                      [a + b + x, 2.0 * modul],
                      [2.0 * a + b + x, -c],
                      [pi * mx + x, -c],
                      [pi * mx + x, modul - height],
                      [0.0 + x, modul - height]]
    
    res = cq.Workplane('XY')
    res = res.polyline(underside_points).close()
    res = res.workplane(offset=width)
    res = res.polyline(topside_points).close()
    res = res.loft()
    
    teeth = copier(res, (1.0, 0.0, 0.0), nz, pi * mx, 0.0)
    res = teeth[0]
    for t in teeth[1:]:
        res = res.union(t)
        
    box = cq.Workplane().box(length, height + modul + 1.0, width + 1.0, centered=False)
    box = box.translate((abs(x), -height + modul - 0.5, -0.5))
    res = res.intersect(box)
    res = res.translate([-pi * mx * (nz - 1.0) / 2.0 - a - b / 2.0, -modul, 0.0])

    return res

show_object(rack(modul=1.0, length=60.0, height=5.0, width=20.0, pressure_angle=20.0, helix_angle=0.0))

In [ ]:

def mountable_rack(modul, length, height, width, pressure_angle, helix_angle, fasteners, profile, head):
    a_rack = rack(modul, length, height, width, pressure_angle, helix_angle)
    offset = length / fasteners;

    def ph_screw(idx):
        screw = cq.Workplane('XZ')
        screw = screw.circle(profile).extrude(profile * 0.6 + modul * 2.25)
        screw = screw.faces('>Y').circle(profile / 2.0).extrude(height + modul)
        screw = screw.translate((idx * offset + (offset - length) / 2.0, modul, width / 2.0))
        return screw
    
    def cs_screw(idx):
        screw = cq.Workplane('XZ')
        screw = screw.circle(profile * 1.5 / 2.0).extrude(profile * 1.25 + modul * 2.25)
        screw = screw.faces('>Y').circle(profile / 2.0).extrude(height + modul)
        screw = screw.translate((idx * offset + (offset - length) / 2.0, modul, width / 2.0))
        return screw
    
    def c_screw(idx):
        screw = cq.Workplane('XZ').workplane(modul * 2.25)
        screw = screw.circle(profile).workplane(offset=profile / 2.0)
        screw = screw.circle(profile / 2.0).loft()
        screw = screw.faces('>Y').circle(profile).extrude(-modul * 2.25)
        screw = screw.faces('<Y').circle(profile / 2.0).extrude(height + modul)
        screw = screw.translate((idx * offset + (offset - length) / 2.0, modul, width / 2.0))
        return screw
    
    def rc_screw(idx):
        screw = cq.Workplane('XZ').workplane(modul * 2.25 + profile / 4.0)
        screw = screw.circle(profile).workplane(offset=profile / 2.0)
        screw = screw.circle(profile / 2.0).loft()
        screw = screw.faces('>Y').circle(profile).extrude(-modul * 2.25 - profile / 4.0)
        screw = screw.faces('<Y').circle(profile / 2.0).extrude(height + modul)
        screw = screw.translate((idx * offset + (offset - length) / 2.0, modul, width / 2.0))
        return screw
    
    def css_screw(idx):
        screw = cq.Workplane('XZ').workplane(modul * 2.25)
        screw = screw.circle(profile).workplane(offset=profile * 0.6)
        screw = screw.circle(profile / 2.0).loft()
        screw = screw.faces('>Y').circle(profile).extrude(-modul * 2.25)
        screw = screw.faces('<Y').circle(profile / 2.0).extrude(height + modul)
        screw = screw.translate((idx * offset + (offset - length) / 2.0, modul, width / 2.0))
        return screw
    
    screw_f = {
        'PH': ph_screw,
        'CS': cs_screw,
        'C': c_screw,
        'RC': rc_screw,
        'CSS': css_screw,
    }[head]
    
    for i in range(fasteners):
        a_rack = a_rack.cut(screw_f(i))
    
    return a_rack

show_object(mountable_rack(modul=1.0, length=60.0, height=10.0, width=20.0,
                           pressure_angle=20.0, helix_angle=20.0, profile=3.0, head='CSS',fasteners=3))

In [20]:

def involute_gear_profile(module, teeth_number, pressure_angle=20.0, clearance=0.0, plane=None):
    curve_points = 20
    m = module
    z = teeth_number
    a0 = np.radians(pressure_angle)
    c = clearance

    d0 = m * z         # pitch diameter
    da = m * (z + 2.0) # addendum circle diameter
    dd = m * (z - 2.0) - 2.0 * c # dedendum circle diameter
    s0 = m * (np.pi / 2.0) # tooth thickness on the pitch circle
    inv_a0 = tan(a0) - a0 # pressure angle involute

    r0 = d0 / 2.0 # pitch circle radius
    ra = da / 2.0 # addendum circle radius
    rd = dd / 2.0 # dedendum circle radius
    rb = np.cos(a0) * r0 # base circle radius
    
    tau = np.pi * 2.0 / z
    
    def calc_phi(r):
        cos_a = r0 / r * np.cos(a0)
        a = np.arccos(cos_a)
        inv_a = np.tan(a) - a
        s = r * (s0 / d0 + inv_a0 - inv_a)
        return s / r
    
    def rotate_point(x, y, angle):
        rx = x * np.cos(angle) - y * np.sin(angle)
        ry = x * np.sin(angle) + y * np.cos(angle)
        return rx, ry
    
    def involute_curve(r, angle, mirror):        
        phi = calc_phi(r)
        if mirror:
            phi = -phi
        x = np.cos(phi) * r
        y = np.sin(phi) * r
        
        return rotate_point(x, y, angle)
    
    if plane is None:
        profile = cq.Workplane('XY')
    else:
        profile = plane
        
    rr = max(rb, rd)
    
    angle = 0.0
    for i in range(z):
        
        profile = profile.parametricCurve(lambda r: involute_curve(r, angle, False),
                                          curve_points, rr, ra)
        arc_sp = involute_curve(ra, angle, False)
        arc_ep = involute_curve(ra, angle, True)
        
        profile = profile.moveTo(*arc_sp).radiusArc(arc_ep, ra)
        profile = profile.parametricCurve(lambda r: involute_curve(r, angle, True),
                                          curve_points, rr, ra)
        
        arc_sp = involute_curve(rr, angle, True)
        rho = angle - tau * 0.5
        arc_mp = (np.cos(rho) * rd, np.sin(rho) * rd)
        arc_ep = involute_curve(rr, angle - tau, False)
        
        profile = profile.moveTo(*arc_sp).threePointArc(arc_mp, arc_ep)
        
        angle -= tau

    profile = profile.consolidateWires()
        
    return profile

profile = involute_gear_profile(1.0, 8)
# profile = profile.twistExtrude(10.0, 150.0)
# profile = profile.extrude(10.0)

show(profile)


| | Object: Group
| | | discretize time:    0.00 sec
| | | edge list:        0.01 sec
| | shape render time:    0.01 sec
| overall render time:    0.01 sec
| create bounding box:    0.00 sec
| configure view:       0.04 sec
add shapes:             0.06 sec
configure display:      0.01 sec
Done, using side car 'Gears'


In [25]:
def spur_gear(module, teeth_number, width, bore, pressure_angle=20.0, helix_angle=0.0, optimized=True, herringbone=False):
    # Dimension Calculations
    helix_angle = radians(helix_angle)    
    d = module * teeth_number                               # Pitch Circle Diameter
    r = d / 2                                               # Pitch Circle Radius
    c = 0.0 if teeth_number < 3 else module / 6.0           # Tip Clearance
    df = d - 2.0 * (module + c)                             # Root Circle Diameter
    rf = df / 2.0                                           # Root Radius
    gamma = width / (r * np.tan(np.pi / 2.0 - helix_angle)) # Torsion Angle for Extrusion

    r_hole = (2.0 * rf - bore) / 8.0                        # Radius of Holes for Material-/Weight-Saving
    rm = bore / 2.0 + 2.0 * r_hole                          # Distance of the Axes of the Holes from the Main Axis
    z_hole = floor(2 * pi * rm / (3.0 * r_hole))            # Number of Holes for Material-/Weight-Saving
    
    profile = involute_gear_profile(module, teeth_number, pressure_angle)
    if herringbone:
        res = profile.twistExtrude(width / 2.0, np.degrees(gamma))
        other_half = res.mirror('XY', (0.0, 0.0, width / 2.0))
        res = res.union(other_half)
    else:
        res = profile.twistExtrude(width, np.degrees(gamma)) 

    # "bore"
    res = res.faces('<Z').circle(rm + r_hole * 1.49).cutThruAll()
    
    body = cq.Workplane('XY')
    if optimized:
        res = (res.faces('<Z').
               circle((bore + r_hole) / 2.0).
               circle(bore / 2.0).
               extrude(width))
        
        if (width - r_hole / 2.0) < (width * 2.0 / 3.0):
            ht = width * 2.0 / 3.0
        else:
            ht = width - r_hole / 2.0
    
        body = body.circle(rm + r_hole * 1.51)
        body = body.circle((bore + r_hole) / 2.0)
        for i in range(z_hole):
            pos = sphere_to_cartesian((rm, pi / 2.0, i * pi * 2.0 / z_hole))
            body = body.moveTo(pos[0], pos[1]).circle(r_hole)
        body = body.consolidateWires().extrude(ht)
    else:
        body = body.circle(rm + r_hole * 1.51)
        body = body.circle(bore / 2.0)
        body = body.consolidateWires().extrude(width)

    res = res.union(body)
    return res

def herringbone_gear(module, teeth_number, width, bore, pressure_angle=20.0,
                      helix_angle=0.0, optimized=True):
    return spur_gear(module, teeth_number, width, bore, pressure_angle,
                     helix_angle, optimized, herringbone=True)

gear = spur_gear(module=1.0, teeth_number=60, width=10.0, bore=5.0, pressure_angle=20.0, helix_angle=20.0, optimized=True)
show(gear)


| | Object: Group
| | | (Caching 140306554226672)
| | | build mesh time:    0.85 sec
| | | discretize time:    0.03 sec
| | | edge list:        0.02 sec
| | shape render time:    0.91 sec
| overall render time:    0.92 sec
| create bounding box:    0.26 sec
| configure view:       0.04 sec
add shapes:             1.21 sec
configure display:      0.01 sec
Done, using side car 'Gears'


In [23]:
gear = herringbone_gear(module=1.0, teeth_number=42, width=50.0, bore=5.0, pressure_angle=20.0, helix_angle=20.0, optimized=True)
show(gear)

| | Object: Group
| | | (Caching 140306549283504)
| | | build mesh time:    1.61 sec
| | | discretize time:    0.08 sec
| | | edge list:        0.05 sec
| | shape render time:    1.75 sec
| overall render time:    1.76 sec
| create bounding box:    0.88 sec
| configure view:       0.04 sec
add shapes:             2.67 sec
configure display:      0.01 sec
Done, using side car 'Gears'


In [10]:
plane = cq.Workplane('XY')

c1 = involute_gear_profile(1.5, 12)
c2 = involute_gear_profile(0.5, 12, plane=c1.workplane(10.0).transformed(rotate=(0.0, 0.0, 7)))

c2.loft(ruled=True)

| | Object: Group
| | | (Caching 140306566820272)
| | | build mesh time:    0.38 sec
| | | discretize time:    0.01 sec
| | | edge list:        0.01 sec
| | shape render time:    0.39 sec
| overall render time:    0.40 sec
| create bounding box:    0.00 sec
| configure view:       0.04 sec
add shapes:             0.44 sec
configure display:      0.01 sec
Done, using side car 'Gears'


In [162]:
360 / 24.0

15.0

In [26]:
plane = cq.Workplane('XY')
plane = plane.rect(2.5, 2.5).workplane(0.5).transformed(rotate=(0.0, 0.0, 45)).rect(2.0, 2.0)
plane = plane.workplane(0.5).transformed(rotate=(0.0, 0.0, 45)).rect(1.5, 1.5)
plane = plane.workplane(0.5).transformed(rotate=(0.0, 0.0, 45)).rect(1.0, 1.5)
plane = plane.workplane(0.5).transformed(rotate=(0.0, 0.0, 45)).rect(0.5, 0.5)
plane = plane.loft()
plane

| | Object: Group
| | | (Caching 140306529072688)
| | | build mesh time:    1.66 sec
| | | discretize time:    0.00 sec
| | | edge list:        0.00 sec
| | shape render time:    1.66 sec
| overall render time:    1.67 sec
| create bounding box:    0.00 sec
| configure view:       0.04 sec
add shapes:             1.71 sec
configure display:      0.01 sec
Done, using side car 'Gears'


In [41]:
def senv(theta0, theta):
    res = 1.0 / np.sin(theta0) * np.arccos(np.cos(theta) / np.cos(theta0))
    res -= np.arccos(np.tan(theta0) / np.tan(theta))
    return res

def sphere_to_cartesian(r, theta, phi):
    return (r * np.sin(theta) * np.cos(phi),
            r * np.sin(theta) * np.sin(phi),
            r * -np.cos(theta))

def bevel_involute_gear_profile(module, teeth_number, pressure_angle=20.0, partial_cone_angle=45.0, helix_angle=0.0, clearance=0.0, plane=None):
    partial_cone_angle = np.radians(partial_cone_angle)
    pressure_angle = np.radians(pressure_angle)
    helix_angle = np.radians(helix_angle)
    curve_points = 20
    
    d_outside = module * teeth_number;                # Part Cone Diameter at the Cone Base,
                                                      # corresponds to the Chord in a Spherical Section
    r_outside = d_outside / 2.0
    rg_outside = r_outside / np.sin(partial_cone_angle) # Large-Cone Radius for Outside-Tooth, corresponds to the Length of the Cone-Flank;
#     rg_inside = rg_outside - tooth_width                # Large-Cone Radius for Inside-Tooth  
#     r_inside = r_outside * rg_inside/rg_outside
    c = clearance                                     # Tip Clearance
    df_outside = d_outside - (module + c) * 2.0 * np.cos(partial_cone_angle)
    rf_outside = df_outside / 2.0

    da_outside = d_outside + module * np.cos(partial_cone_angle) * 2.0
    
    if module < 1.0:
        da_outside *= 1.1

    ra_outside = da_outside / 2.0
    delta_a = np.arcsin(ra_outside / rg_outside)

    alpha_spur = np.arctan(np.tan(pressure_angle) / np.cos(helix_angle)) # Helix Angle in Transverse Section
    delta_b = np.arcsin(np.cos(alpha_spur) * np.sin(partial_cone_angle)) # Base Cone Angle  
    delta_f = np.arcsin(rf_outside / rg_outside)

    phi_r = senv(delta_b, partial_cone_angle) # Angle to Point of Involute on Partial Cone
    mirrpoint = np.pi / teeth_number + 2.0 * phi_r

    height_f = rg_outside * np.cos(delta_f)

    start = max(delta_b, delta_f)

    step = (delta_a - delta_b) / 16.0;
    
    
    
    if plane is None:
        profile = cq.Workplane('XY')
    else:
        profile = plane
        
    
    def involute_curve(theta, angle, mirror):
        phi = senv(delta_b, theta)
        r = height_f / np.cos(theta)
        if mirror:
            x, y, z = sphere_to_cartesian(r, theta, mirrpoint - phi)
        else:
            x, y, z = sphere_to_cartesian(r, theta, phi)
        
        return x, y

    
    
    profile = profile.parametricCurve(lambda t: involute_curve(t, 0.0, False),
                                      curve_points, start, delta_a)
    profile = profile.parametricCurve(lambda t: involute_curve(t, 0.0, True),
                                      curve_points, start, delta_a)
    profile = profile.consolidateWires()
    return profile

#     x, y, z = np.ndarray(0), np.ndarray(0), np.ndarray(0)
#     px, py, pz = np.ndarray(0), np.ndarray(0), np.ndarray(0)

#     for tau in np.linspace(0.0, np.pi * 2.0, teeth_number):
#         delta = np.arange(start, delta_a, step)
#         phis = senv(delta_b, delta)
#         rs = np.full(delta.shape[0], rg_outside)

#         x1, y1, z1 = sphere_to_cartesian(rs, delta, phis + tau)
#         x2, y2, z2 = sphere_to_cartesian(rs, delta, mirrpoint - phis + tau)

#         x = np.concatenate((x, x1, x2[::-1]))
#         y = np.concatenate((y, y1, y2[::-1]))
#         z = np.concatenate((z, z1 + height_f, z2[::-1] + height_f))

#         rp = height_f / np.cos(delta)

#         px1, py1, pz1 = sphere_to_cartesian(rp, delta, phis + tau)
#         px2, py2, pz2 = sphere_to_cartesian(rp, delta, mirrpoint - phis + tau)

#         px = np.concatenate((px, px1, px2[::-1]))
#         py = np.concatenate((py, py1, py2[::-1]))
#         pz = np.concatenate((pz, pz1 + height_f, pz2[::-1] + height_f))

prof = bevel_involute_gear_profile(1.0, 18, pressure_angle=20.0, partial_cone_angle=45.0)
show(prof)

| | Object: Group
| | | discretize time:    0.00 sec
| | | edge list:        0.01 sec
| | shape render time:    0.01 sec
| overall render time:    0.01 sec
| create bounding box:    0.00 sec
| configure view:       0.04 sec
add shapes:             0.05 sec
configure display:      0.01 sec
Done, using side car 'Gears'


In [27]:
2.2 / 2.0

1.1

In [69]:
def curve(t, r, a):
    x = np.cos(a + t * 1.0) * r
    y = np.sin(a + t * 1.0) * r
    z = t * 3.0
    return x, y, z
    
c1 = cq.Workplane('XY').parametricCurve(lambda t: curve(t, 2.0, 0.0), 20, 0.0, np.pi).val()
c2 = cq.Workplane('XY').parametricCurve(lambda t: curve(t, 2.0, np.pi / 2.0), 20, 0.0, np.pi).val()
c3 = cq.Workplane('XY').parametricCurve(lambda t: curve(t, 2.0, np.pi), 20, 0.0, np.pi).val()
c4 = cq.Workplane('XY').parametricCurve(lambda t: curve(t, 2.0, np.pi / 2.0 * 3.0), 20, 0.0, np.pi).val()



f1 = cq.Face.makeRuledSurface(c1, c2)
f2 = cq.Face.makeRuledSurface(c2, c3)
f3 = cq.Face.makeRuledSurface(c3, c4)
f4 = cq.Face.makeRuledSurface(c4, c1)

sh = cq.Shell.makeShell([f1, f2, f3, f4])
body = cq.Solid.makeSolid(sh)

show(body)
body.exportStl('/home/michael/Downloads/twisted.stl')

| | Object: Group
| | | (Caching 140305864175472)
| | | build mesh time:    0.28 sec
| | | discretize time:    0.00 sec
| | | edge list:        0.00 sec
| | shape render time:    0.29 sec
| overall render time:    0.29 sec
| create bounding box:    0.00 sec
| configure view:       0.03 sec
add shapes:             0.33 sec
configure display:      0.01 sec
Done, using side car 'Gears'


True

In [75]:
c1 = (cq.Workplane('XY')
      .parametricCurve(lambda t: curve(t, 2.0, 0.0), 20, 0.0, np.pi)
      .val())

c2 = (cq.Workplane('XY')
      .parametricCurve(lambda t: curve(t, 2.0, np.pi), 20, 0.0, np.pi)
      .val())
f1 = cq.Face.makeRuledSurface(c1, c2)

show(f1)

| | Object: Group
| | | (Caching 140305896654832)
| | | build mesh time:    0.10 sec
| | | discretize time:    0.00 sec
| | | edge list:        0.01 sec
| | shape render time:    0.10 sec
| overall render time:    0.11 sec
| create bounding box:    0.00 sec
| configure view:       0.05 sec
add shapes:             0.16 sec
configure display:      0.01 sec
Done, using side car 'Gears'


In [7]:
from OCP.Precision import Precision
from OCP.TColgp import TColgp_HArray1OfPnt, TColgp_HArray2OfPnt
from OCP.GeomAPI import (
    GeomAPI_Interpolate,
    GeomAPI_ProjectPointOnSurf,
    GeomAPI_PointsToBSpline,
    GeomAPI_PointsToBSplineSurface,
)
from OCP.BRepBuilderAPI import (
    BRepBuilderAPI_MakeVertex,
    BRepBuilderAPI_MakeEdge,
    BRepBuilderAPI_MakeFace,
    BRepBuilderAPI_MakePolygon,
    BRepBuilderAPI_MakeWire,
    BRepBuilderAPI_Sewing,
    BRepBuilderAPI_Copy,
    BRepBuilderAPI_GTransform,
    BRepBuilderAPI_Transform,
    BRepBuilderAPI_Transformed,
    BRepBuilderAPI_RightCorner,
    BRepBuilderAPI_RoundCorner,
    BRepBuilderAPI_MakeSolid,
)

def makeSplineApprox(points, tol=1e-2, smoothing=False, minDeg=1, maxDeg=3):
        points_ = TColgp_HArray2OfPnt(1, len(points), 1, len(points[0]))

        for i, vi in enumerate(points):
            for j, v in enumerate(vi):
                v = cq.Vector(*v)
                points_.SetValue(i + 1, j + 1, v.toPnt())

        if smoothing:
            spline_builder = GeomAPI_PointsToBSplineSurface(
                points_, *smoothing, DegMax=maxDeg, Tol3D=tol
            )
        else:
            spline_builder = GeomAPI_PointsToBSplineSurface(
                points_, DegMin=minDeg, DegMax=maxDeg, Tol3D=tol
            )

        if not spline_builder.IsDone():
            raise ValueError("B-spline approximation failed")

        spline_geom = spline_builder.Surface()

        return cq.Face(BRepBuilderAPI_MakeFace(spline_geom, Precision.Confusion_s()).Face())

In [8]:
r = 2.0

t = np.linspace(0.0, np.pi, 20)
x1 = np.cos(t * 2.0) * r
y1 = np.sin(t * 2.0) * r
z1 = t * 3.0

x2 = np.cos(np.pi / 2.0 + t * 2.0) * r
y2 = np.sin(np.pi / 2.0 + t * 2.0) * r
z2 = t * 3.0


p1 = np.dstack((x1, y1, z1)).squeeze()
p2 = np.dstack((x2, y2, z2)).squeeze()

grid = np.stack((p1, p2))

f1 = makeSplineApprox(grid)
f2 = f1.rotate(cq.Vector(0, 0, 0), cq.Vector(0, 0, 1), 90.0)
f3 = f1.rotate(cq.Vector(0, 0, 0), cq.Vector(0, 0, 1), 180.0)
f4 = f1.rotate(cq.Vector(0, 0, 0), cq.Vector(0, 0, 1), 270.0)

bottom = (cq.Workplane('XY')
          .rect(np.sqrt(2 * r ** 2), np.sqrt(2 * r ** 2))
          .rotate(cq.Vector(0, 0, 0), cq.Vector(0, 0, 1), 45.0)).val()
top = bottom.translate(cq.Vector(0.0, 0.0, np.pi * 3.0))
top = cq.Face.makeFromWires(top)
bottom = cq.Face.makeFromWires(bottom)

sh = cq.Shell.makeShell([bottom, f1, f2, f3, f4, top])
body = cq.Solid.makeSolid(sh)

body = (cq.Workplane('XZ')
        .add(body)
        .moveTo(0.0, np.pi * 3.0 / 2.0)
        .circle(0.8)
        .cutThruAll())

show(body)


body.val().exportStl('/home/michael/Downloads/twisted.stl')

| | Object: Group
| | | (Caching 140095567492400)
| | | build mesh time:    1.26 sec
| | | discretize time:    0.00 sec
| | | edge list:        0.01 sec
| | shape render time:    1.27 sec
| overall render time:    1.28 sec
| create bounding box:    0.00 sec
| configure view:       0.04 sec
add shapes:             1.32 sec
configure display:      0.04 sec
Done, using side car 'Gears'


True

In [122]:
v = cq.Vector(*grid[1, 0])
v
# v.toPnt()

Vector: (-2.0, 2.4492935982947064e-16, 0.0)

In [89]:
cq.__version__

'2.1'

In [ ]:
TColgp_HArray2OfPnt